# Random forest lepton flavor study
(Oct 26, 2016)

This study was done using the the same, very simliar, set-up and methodology to what was done with TMVA over during 2015-mid2016. There are a few glaring mistakes and over sights which will be noted but not corrected in this study. The issues are outlined below. 

### Issues 

There are a few issues to note. First that the original forest was not trained on 1/3 of the mc, but ~ 1/3 of WW and the rest of the same order as the WW.  ( 50,000-WW, 70,000-DY, 33,000-TT ) More importantly when producing yeild tables the entire Monte Carlo dataset set was used, including the portion of the set which had been used for training. What worse is that only ~2/3 of the WW set was avalible. So over 50% of the data set used to test on was also used to train. :( 

Other things: 

- At this point in time bagged fraction isn't known to be a tunable parameter.

- Sklearn seems to cut a bit more harshly on TT then DY as the yields for TT are abit lower while DY is markedly higher then what's produced by TMVA.

- 100 trees were trained for each flavor and for the combined set here.  The TMVA forest held 300 trees.

### Future 

I plan to move away from TMVA.  Before doing so I need to determine why their is such a large difference in the DY yields. I plan on completing the analysis with 1/3 of the MC where the number of DY and TT are weighted such that the weighted sum of each process are the same magnitude.

link to that note book: 

=========================


## Set up:

In [ ]:
run ../prep_ana.py

In [ ]:
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%matplotlib inline

In [ ]:
def process_yields( df ):
    for process in df.process.unique():
        for decay in df[df.process == process].process_decay.unique():
            if decay in scales.keys():
                print process, decay, df[(df.process_decay==decay)].shape[0], df[(df.process_decay==decay)].shape[0] * scales[decay]

In [ ]:
# Concate processes and preform preselection
df_train_test = pre_cuts(df)
df_train_test = df_train_test.reset_index()

In [ ]:
process_yields( df_train_test )

In [ ]:
#Select a subset of events that will be used for training and testing

#train_DY = df_train_test[df_train_test.process=="DY"].sample( frac=0.4 )
train_WW = df_train_test[df_train_test.process=="WW"].sample( frac=0.4)#n=train_DY.shape[0]*2 )
train_DY = df_train_test[df_train_test.process=="DY"].sample( n=train_WW.shape[0]*2  )#.45
train_TT = df_train_test[df_train_test.process=="Top"].sample( n=train_WW.shape[0]*2 )

train = pd.concat( [train_WW, train_TT, train_DY] )

train_same = train[ train.lep_Type < 0]
train_diff = train[ train.lep_Type > 0]

In [ ]:
#Set up test sets

test_same = df_train_test[ df_train_test.lep_Type < 0].drop( train_same.index )
test_diff = df_train_test[ df_train_test.lep_Type > 0].drop( train_diff.index )

scales_test = {key: scales[key] * ( float(df_train_test[ df_train_test.process_decay == key].shape[0])/ float(test_same[ test_same.process_decay == key].shape[0] + test_diff[ test_diff.process_decay == key].shape[0]) ) for key in scales.keys() if key in df_train_test.process_decay.unique() } 

In [ ]:
df_da_a = rp.read_root(data_path+"/muon_2012A_complete.root")
df_da_b = rp.read_root(data_path+"/muon_2012B_complete.root")
df_da_c = rp.read_root(data_path+"/muon_2012C_complete.root")
df_da_d = rp.read_root(data_path+"/muon_2012D_complete.root")

df_da = pd.concat([df_da_a, df_da_b, df_da_c, df_da_d])

In [ ]:
df_da_pre = pre_cuts(df_da)
df_da_same = df_da_pre[ df_da_pre.lep_Type < 0]
df_da_diff = df_da_pre[ df_da_pre.lep_Type > 0]

# Features

In [ ]:
features = ['qT',\
            'dPhiLL', 'HT', 'recoil']

# Classification training and testing

In [ ]:
#Create labels 
#labels same
labels_same = np.empty( train_same.shape[0] )
weights_same = np.empty( train_same.shape[0])

labels_same[(train_same.process == "WW").as_matrix()] = 1
labels_same[(train_same.process == "Top").as_matrix()] = 2
labels_same[(train_same.process == "DY").as_matrix()] = 2

weights_same[(train_same.process == "WW").as_matrix()] = 1
weights_same[(train_same.process == "Top").as_matrix()] = 1
weights_same[(train_same.process == "DY").as_matrix()] = 1


#labels diff
labels_diff = np.empty( train_diff.shape[0] )
weights_diff = np.empty( train_diff.shape[0])

labels_diff[(train_diff.process == "WW").as_matrix()] = 1
labels_diff[(train_diff.process == "DY").as_matrix()] = 2
labels_diff[(train_diff.process == "Top").as_matrix()] = 2

weights_diff[(train_diff.process == "WW").as_matrix()] = 1
weights_diff[(train_diff.process == "DY").as_matrix()] = 1
weights_same[(train_same.process == "Top").as_matrix()] = 1


## Different flavor 

In [ ]:
clf_d = RandomForestClassifier(n_estimators=50, n_jobs=-1, min_samples_split=10, max_depth=15, max_features='sqrt')
clf_s = RandomForestClassifier(n_estimators=50, n_jobs=-1, min_samples_split=10, max_depth=15, max_features='sqrt')
#n_estimators=150

In [ ]:
clf_d = clf_d.fit( np.float32(train_diff[features].values) , np.float32(labels_diff), sample_weight=weights_diff)

In [ ]:
print clf_d.score(np.float32(train_diff[features].values), np.float32(labels_diff), weights_diff)

In [ ]:
pred_d = clf_d.predict_proba(np.float32(test_diff[features].values))
test_diff["pred"] = pred_d[:,0]

In [ ]:
pred_d = clf_d.predict_proba(np.float32(df_da_diff[features].values))
df_da_diff["pred"] = pred_d[:,0]

In [ ]:
bins_mc = bin_df( test_diff[ test_diff.mll > 30], "pred", range=(0,1), bins=50, scales=scales_test)
bins_da = bin_df( df_da_diff[ df_da_diff.mll > 30], "pred", range=(0,1), bins=50)

full_plot( bins_mc, bins_da, color="color_1", title="different flavor")

## Same Flavor

In [ ]:
clf_s = clf_s.fit( np.float32(train_same[features].values) , np.float32(labels_same), sample_weight=weights_same)

In [ ]:
print clf_s.score(np.float32(train_same[features].values), np.float32(labels_same), weights_same)

In [ ]:
pred_s = clf_s.predict_proba(np.float32(test_same[features].values))
test_same["pred"] = pred_s[:,0]

In [ ]:
pred_s = clf_s.predict_proba(np.float32(df_da_same[features].values))
df_da_same["pred"] = pred_s[:,0]

In [ ]:
bins_mc = bin_df( test_same[ test_same.mll > 30], "pred", range=(0,1), bins=50, scales=scales_test)
bins_da = bin_df( df_da_same[ df_da_same.mll > 30], "pred", range=(0,1), bins=50)

full_plot( bins_mc, bins_da, color="color_1", title="same flavor")

In [ ]:
bins_mc = bin_df( test_same[ test_same.mll > 30], "mll", scales=scales_test)
bins_da = bin_df( df_da_same[ df_da_same.mll > 30], "mll", )

full_plot( bins_mc, bins_da, color="color_1",)

In [ ]:
bins_mc = bin_df( test_diff[ test_diff.mll > 30], "metMod", scales=scales_test)
bins_da = bin_df( df_da_diff[ df_da_diff.mll > 30], "metMod",)

full_plot( bins_mc, bins_da, color="color_1",)

## same + different flavors

In [ ]:
clf = clf.fit( np.float32(train[features].values) , np.float32(labels), sample_weight=weights)

In [ ]:
print clf.score(np.float32(train[features].values), np.float32(labels), weights)

In [ ]:
pred = clf.predict_proba(np.float32(df[features].values))

In [ ]:
df["pred"] = pred[:,0]

In [ ]:
df[ df.WW == 1 ].pred.hist(bins=1000)

In [ ]:

scales = {"WW": 19.7e3 * 59.8 /  10000431.0 ,\
          "DY": 19.7e3 * 3531.9 / 30459500. ,\
          "TT": 19.7e3 * 25.81 / 12011428.,\
          "ZZ": 19.7e3 * 9.03 / 9799908.,\
          "WZ": 19.7e3 * 1.07 / 2017979., "WJ": 1 }


for process in scales.keys():
    if process in df.keys():
        print process, df[ (df[process] == 1) & (df.pred > .96912) & (df.lep_Type > 0)].shape[0] *  scales[process],\
        df[ (df[process] == 1) & (df.pred > .96912) & (df.lep_Type < 0)].shape[0] *  scales[process]

In [ ]:
df.shape

# Results as of sept 21 

### trained seperately:

Process:    | WW | DY | TT | ZZ+WZ
---         |---| ---|--- |---
same flavor |1684|395|71 | 192 
diff flavor |3799|661|139 | 220 

Significant differences

- diff DY 661 vs 106 
- diff TT 139 vs 433


### trained together:

Process:    | WW | DY | TT | ZZ+WZ
---         |---| ---|--- |---
same flavor |1057|182|36 | 101 
diff flavor |4426|485|178 | 279 

# End

# WW practice selection


In [ ]:
    a = 1. * df_train_test.METProj_sin.values
    a[df_train_test.process.values == "DY"] = df_train_test.METProj_sin.values[df_train_test.process.values == "DY"] * 1.059
    df_train_test["METProj_sin"] = a

In [ ]:
def process_yields_v2( df ):
    tot = 0
    for process in df.process.unique():
        sum_process = 0
        for decay in df[df.process == process].process_decay.unique():
            if decay in scales.keys():
                #print process, decay, df[(df.process_decay==decay)].shape[0], df[(df.process_decay==decay)].shape[0] * scales[decay]
                sum_process += df[(df.process_decay==decay)].shape[0] * scales[decay]
        print process, sum_process
        tot += sum_process
    print "tot",tot

In [ ]:
def WW_same( df ):
  initial_cuts = (df.mll > 30 )  & (df.numbExtraLep == 0) & (df.numb_jets <  3) & ( df.metMod > 50 ) & (df.mllMET > 100) & (df.HT < 50) & (df.qT > 35)

  basic_df_0j_cuts = initial_cuts & (df.lep_Type < 0) & (df.METProj_sin > 50)
  return df[basic_df_0j_cuts]

In [ ]:
process_yields_v2( WW_same(df_train_test) )

In [ ]:
bins_mc = bin_df( WW_same(df_train_test), "dPhiLL",)
bins_da = bin_df( WW_same(df_da_pre), "dPhiLL",)

full_plot( bins_mc, bins_da, color="color_1", logy=False, y_range=(0,np.max(bins_da["Da"][0])+50))

In [ ]:
def WW_diff( df ):
  initial_cuts = (df.mll > 30 )  & (df.numbExtraLep == 0) & (df.numb_jets <  3) & ( df.metMod > 50 ) & (df.mllMET > 100) & (df.HT < 50) & ( df.metMod < 120 )

  basic_df_0j_cuts = initial_cuts & (df.lep_Type > 0) & (df.METProj_sin > 30)
  return df[basic_df_0j_cuts]

In [ ]:
process_yields_v2( WW_diff(df_train_test) )

In [ ]:
bins_mc = bin_df( WW_diff(df_train_test), "metMod",)
bins_da = bin_df( WW_diff(df_da_pre), "metMod",)

full_plot( bins_mc, bins_da, color="color_1", logy=False, y_range=(0,np.max(bins_da["Da"][0])+50))

In [ ]:
def calc_cross( df_mc, df_da ):
    sum_events = 0
    for key in scales:
        if key in df_mc.process_decay.unique() and key != 'WW':
            sum_events += df_mc[df_mc.process_decay == key].shape[0] * scales[key]
    LBrEff = 19.7e3 * (3*.108)**2 *.122
    print df_mc[df_mc.process == 'WW'].shape[0] * scales["WW"] / (LBrEff * df_mc[df_mc.process == 'WW'].shape[0] / (df[df.process =='WW'].shape[0]) )
    return (df_da.shape[0] - sum_events -217- 447 ) / (LBrEff * df_mc[df_mc.process == 'WW'].shape[0] / (df[df.process =='WW'].shape[0]) )

In [ ]:
calc_cross( pd.concat( [WW_diff(df_train_test), WW_same(df_train_test)]), pd.concat([WW_diff(df_da_pre), WW_same(df_da_pre)]) ) 

In [ ]:
calc_cross( WW_diff(df_train_test), WW_diff(df_da_pre) ) 

In [ ]:
calc_cross(WW_same(df_train_test), WW_same(df_da_pre))

In [ ]:
189+81+114+63

In [ ]:
(831.59 + 2684.7 - 823 - 2213 ) / ( 1031.59 + 2884.7)

In [ ]:
df_train_test.process_decay.unique()

In [ ]:
( 831.59 + 2684.7)/ (19.7e3 * (3*.108)**2 *.122 * 3000 / (df[df.process =='WW'].shape[0]*scales['WW']) )

#b# 

In [ ]:
df.keys()

In [ ]:
yields_test = process_yields_v3( pd.concat( [WW_diff(df_train_test), WW_same(df_train_test)]) ) 

In [ ]:
from IPython.core.display import display, HTML
display(HTML(yields_test.to_html(columns=["Process", "Same Flavor", "Diff Flavor"], index=False)))


In [ ]:
save_df_to_html(yields_test, "another_test.md")